# Newton Divided Difference

In [1]:
import lib_path
from smalllab.table import *
from IPython.display import HTML
from math import cos

## Divided Differences

Let $x_0$ ad $x_1$ be distinct numbers. Define the first order divided difference of $f(x)$ as
$$
f[x_0,x_1] = \frac{f(x_1)-f(x_0)}{x_1-x_0}.
$$

In [2]:
def divdif_1(f, x0, x1):
    return (f(x1)-f(x0)) / (x1-x0)

In [3]:
f = lambda x: cos(x)
x0, x1 = 0.2, 0.3

divdif_1(f, x0, x1)

-0.2473008871563565

### Higher order divided difference

Let $x_0,x_1,\ldots,x_n$ be $n+1$ distinct numbers.
Define the higher order divided difference as
$$
f[x_0,\ldots,x_n] = \frac{f[x_1,\ldots,x_n]-f[x_0,\ldots,x_{n-1}]}{x_n-x_0}.
$$

In [4]:
f = lambda x: cos(x)
x0, x1, x2 = 0.2, 0.3, 0.4

d1 = divdif_1(f, x1, x2)
d2 = divdif_1(f, x0, x1)
(d1-d2) / (x2-x0)

-0.4772703203542608

In [5]:
def divdif(X, Y) -> list[float]:
    """Calculate the set of divided differences
    f[x0,x1], f[x0,x1,x2], ..., f[x0,x1,...,xn].
    
    Parameters
    ----------
    X, Y : list of int or float
        All entries in X must be distinct.
        The length of X and Y must be equal and greater than 1.
    """
    if len(X) != len(Y):
        raise ValueError("the length of X and Y must be equal")
    if len(X) < 2:
        raise ValueError("the length of X and Y must be greater than 1")
    if len(set(X)) != len(X):
        raise ValueError("all values in X must be distinct")
        
    n = len(X)
    D = Y.copy()
    for j in range(1, n):
        for k in range(j, n):
            D[k] = (D[k]-D[j-1]) / (X[k]-X[j-1])
    return D

In [6]:
f = lambda x: cos(x)
X = [0.2, 0.3, 0.4]
Y = [f(j) for j in X]

divdif(X, Y)

[0.9800665778412416, -0.2473008871563565, -0.4772703203542609]

In [7]:
f = lambda x: cos(x)
index = [0, 1, 2, 3, 4, 5, 6]
X = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2]
Y = [f(j) for j in X]
D = divdif(X, Y)
head = ["$i$", "$x_i$", "$\cos(x_i)$", "$D_i$"]

T = table_v(head, index, X, Y, D)
HTML(T)

$i$,$x_i$,$\cos(x_i)$,$D_i$
0,0.0,1.0,1.0
1,0.2,0.9800665778412416,-0.09966711079379187
2,0.4,0.9210609940028851,-0.48840202099497687
3,0.6,0.8253356149096783,0.04900763384891428
4,0.8,0.6967067093471654,0.038122457312403296
5,1.0,0.5403023058681398,-0.00396204670695722
6,1.2,0.3623577544766736,-0.001134889809527827


## Newton's Divided Difference Interpolation

$$
P_n(x) = f(x_0) + (x-x_0)f[x_0,x_1] + (x-x_0)(x-x_1)f[x_0,x_1,x_2] +\cdots +
(x-x_0)(x-x_1)\cdots (x-x_{n-1})f[x_0,x_1,\ldots,x_n].
$$

Rewrite the above formula as
$$
P_n(x) = D_0+(x-x_0)D_1+(x-x_0)(x-x_1)D_2+\cdots+(x-x_0)(x-x_1)\cdots(x-x_{n-1})D_n
$$
with
$$
D_0=f(x_0), \qquad D_i=f[x_0,\ldots,x_i].
$$

Rewrite in the nested form
$$
P_n(x) = D_0 + (x-x_0)[D_1+(x-x_1)[D_2+\cdots +(x-x_{n-2})[D_{n-1}+(x-x_{n-1})D_n]\cdots].
$$

It can be evaluated using recurrence relations:
\begin{align}
P_0(x) &= D_n,\\
P_k(x) &= D_{n-k} + (x-x_{n-k})P_{k-1}(x), \qquad k=1,2,\ldots n.
\end{align}

In [8]:
def newton_divdif(X, Y, xp) -> float:
    """Evaluate the Newton divided difference at xp."""
    n = len(X) - 1
    D = divdif(X, Y)
    p = D[n]
    for j in range(1, n+1):
        p = D[n-j] + (xp-X[n-j])*p
    return p

In [9]:
f = lambda x: cos(x)
X = [0.0, 0.2, 0.4, 0.6, 0.8, 1.0, 1.2]
Y = [f(j) for j in X]

newton_divdif(X, Y, 0.1)

0.9950040606679764

In [10]:
f(0.1)

0.9950041652780258

In [11]:
newton_divdif(X, Y, 0.3)

0.9553365188727757

In [12]:
f(0.3)

0.955336489125606

In [13]:
newton_divdif(X, Y, 0.5)

0.8775825447048117

In [14]:
f(0.5)

0.8775825618903728

In [15]:
X = [5, 7, 11, 13, 17]
Y = [150, 392, 1452, 2366, 5202]
xp = 9
newton_divdif(X, Y, xp)

810.0

## Reference

- Numerical Methods in Engineering with Python 3 by Jann Kiusalaas. Chapter 3.2.
- Elementary Numerical Analysis 3ed. Kendall Atkinson, Weimin Han. Chapter 3.2.